In [1]:
from IPython.display import HTML

# Cell visibility - COMPLETE:
#tag = HTML('''<style>
#div.input {
#    display:none;
#}
#</style>''')
#display(tag)

#Cell visibility - TOGGLE:
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<p style="text-align:right">
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.</p>''')
display(tag)

## Függvénygörbék

Az interaktív példa bemutatja az irányítástechnikában használt alapvető függvényeket, valamint paramétereiknek hatását (frekvencia, amplitúdó, fázis, lecsengés, stb.). A kiválasztott függvény megjelenik az diagram területen a releváns paramétereinek megfelelően (utóbbiak csúszkákkal módosíthatók).
A példa az alábbi függvényeket tartalmazza:
* szinusz hullám,
* koszinusz hullám,
* csillapított hullám,
* impulzus (Dirac delta) függvény,
* egységugrás függvény,
* rámpafüggvény,
* háromszög hullám.

In [2]:
%matplotlib inline
#%config InlineBackend.close_figures=False 
from ipywidgets import interactive
from ipywidgets import widgets
from IPython.display import Latex, display, Markdown # For displaying Markdown and LaTeX code
import matplotlib.pyplot as plt
import numpy as np
import math
import matplotlib.patches as mpatches
from IPython.display import HTML, clear_output
from IPython.display import display


# Functions descriptions
sine_text = "Szinusz hullám: A szinusz hullám egy matematikai görbe, ami egy sima, periodikus rezgést ír le. Egy folyamatos hullám, ami a szinusz függvényről kapta a nevét."
cosine_text = "Koszinusz hullám: A koszinusz hullám alakja hasonló a szinuszéhoz, azzal a különbséggel, hogy 1/4 ciklussal, azaz 90° fázissal a szinum előtt van"
dumped_text = "Csillapított hullám: a csillapított hullám egy szinusz hullám, aminek amplitúdója az idő előrehaladtával fokozatosancsökken, előbb-utóbb elérve a nullát. Ideálisan az amplitúdó az exponenciális függvény mentén csökken."
delta_text = "Impulzus függvény: Matematikailag az impulzus, vagy Dirac delta (δ) függvény egy általánosított függvény, amit Paul Dirac fizikus vezetett be. A függvény alapvetően egy ideális, pontszerű tömeget vagy töltést modellez, ahol a függvényérték az origü kivételével mindenhol nulla, integráltja pedig egységnyi."
step_text = "Egységugrás függvény: Az egységugrás (vagy lépcső) függvény egy elemenként konstans részekből álló függvényalak. Véges számú szakaszból épül fel, amik mindegyike konstans, valós szám értékű."
ramp_text = "Rámpa függvény: A ráma függvény egy egyparaméteres, rámpa alakú függvény. Számos módon definiálható, negatív bemenet esetében a függvényérték 0, egyés esetekben pedig a kimenet a bemenettel egyező."
triang_text = "Háromszög hullám: A háromszög hullám egy nem-szinuszos hullámforma, ami a nevét háromszög alakjáról kapta. Periodikus, elemenként lineáris, folytonos és valós értékű függvény."

# Sine widgets
slider_a = widgets.FloatSlider(description='Amplitúdó', min=0., max=4., step=0.25, continuous_update=False)
slider_f = widgets.FloatSlider(description='Frekvencia', min=0., max=10., step=0.5, continuous_update=False)
slider_p = widgets.FloatSlider(description='Fázis', min=-10.0, max=10.0, step=0.5, continuous_update=False)
slider_f.value = 5 
slider_a.value = 2
slider_p.value = 0 
formula_sine = r'$f(t)= Asin(2{\pi}ft + {\varphi})$'

# Cosine widgets
slider_acos = widgets.FloatSlider(description='Amplitúdó', min=0., max=4., step=0.25, continuous_update=False)
slider_fcos = widgets.FloatSlider(description='Frekvencia', min=0., max=10., step=0.5, continuous_update=False)
slider_pcos = widgets.FloatSlider(description='Fázis', min=-10.0, max=10.0, step=0.5, continuous_update=False)
slider_fcos.value = 5 
slider_acos.value = 2
slider_pcos.value = 0 
formula_cosine = r'$f(t)= Acos(2{\pi}ft + {\varphi})$'

# Damping widgets
slider_adamp = widgets.FloatSlider(description='Amplitúdó', min=0., max=4., step=0.25, continuous_update=False)
slider_fdamp = widgets.FloatSlider(description='Frekvencia', min=0., max=10., step=0.5, continuous_update=False)
slider_pdamp = widgets.FloatSlider(description='Fázis', min=-10.0, max=10.0, step=0.5, continuous_update=False)
slider_d = widgets.FloatSlider(description='Decay', min=0., max=3., step=0.2, continuous_update=False)
slider_fdamp.value = 5 
slider_adamp.value = 2
slider_pdamp.value = 0 
slider_d.value = 0 
formula_damp = r'$f(t)= Ae^{-{\lambda}t}cos(2{\pi}ft + {\varphi})$'

# Delta widgets
slider_adelta = widgets.FloatSlider(description='Paraméter a', value = 0.01, min=0.01, max=1.5, step=0.01, continuous_update=False)
formula = r'$\delta_{a}(x)= \frac{1}{|a|\sqrt{\pi}}e^{-(x/a)^2}$'

# Step widget
formula_step = r'$ f(x) = \begin{cases} b, & \mbox{if } x > a \\ 0, & \mbox{if } x \leq a \end{cases} $'
slider_astep = widgets.FloatSlider(description='Paraméter a', value = 0., min=-10, max=10, step=0.5, continuous_update=False)
slider_bstep = widgets.FloatSlider(description='Paraméter b', value = 1, min=0, max=5., step=0.5, continuous_update=False)

# Ramp widgets
formula_ramp = r'$ f(x) = \begin{cases} x-a, & \mbox{if } x > a \\ 0, & \mbox{if } x \leq a \end{cases} $'
slider_aramp = widgets.FloatSlider(description='Paraméter a', value = 0., min=-10, max=10, step=0.5, continuous_update=False)

# Triangle widgets
slider_atri = widgets.FloatSlider(description='Amplitúdó', min=1, max=4., step=0.5, continuous_update=False)
slider_ptri = widgets.FloatSlider(description='Periódus', min=1, max=10., step=0.5, continuous_update=False)
formula_triangle = r'$f(t)= \frac{2a}{\pi}arcsin(sin(\frac{2\pi}{p}t))$'

# Layouts
info_layout = widgets.Layout(border='solid black', width = '100%', height = '200', padding='5px')
panel_layout = widgets.Layout(border='solid blue', width = '35%', height = '175', padding='5px')
plot_layout = widgets.Layout(border='solid red', width = '65%', height = '175', padding='5px')
output_info = widgets.Output(layout = info_layout)
output_panel = widgets.Output(layout = panel_layout)
output_plot = widgets.Output(layout = plot_layout)

# Dropdown widget
dd_order = widgets.Dropdown(
    options=['szinusz hullám', 'koszinusz hullám', 'csillapított hullám', 'impulzus függvény',
             'egységugrás függvény', 'rámpa függvény', 'háromszög hullám'],
    value='szinusz hullám',
    description='Válasszon függvényt:',
    disabled=False,
    style = {'description_width': 'initial'},
)

# Functions
def f_sin(A, frequency, phase):
    
    plt.figure(figsize=(10,5))
    t = np.linspace(-10, 10, num=1000)
    plt.plot(t, A * np.sin(t*frequency + phase), 'b-')
    plt.xlim(-10, 10)
    plt.ylim(-5, 5)
    plt.grid(True)
    plt.xlabel('t [s]')
    plt.ylabel('$f(t)$')
    plt.axhline(y=0,lw=0.8,color='k')
    plt.axvline(x=0,lw=0.8,color='k')
    with output_plot:
        output_plot.clear_output(wait=True)
        plt.show()

def f_cos(A, frequency, phase):
    plt.figure(figsize=(10,5))
    t = np.linspace(-10, 10, num=1000)
    plt.plot(t, A * np.cos(t*frequency + phase), 'r-')
    plt.xlim(-10, 10)
    plt.ylim(-5, 5)
    plt.grid(True)
    plt.xlabel('t [s]')
    plt.ylabel('$f(t)$')
    plt.axhline(y=0,lw=0.8,color='k')
    plt.axvline(x=0,lw=0.8,color='k')
    with output_plot:
        output_plot.clear_output(wait=True)
        plt.show()
        
def f_damping(A, frequency, phase, decay):
    plt.figure(figsize=(10,5))
    x = np.linspace(0, 10, num=1000)
    #plt.plot(x, [A * math.exp(-decay * t) *(np.cos(t*frequency + phase) + np.sin(t*frequency + phase)) for t in x])
    plt.plot(x, [A * math.exp(-decay * t) *(np.cos(t*frequency + phase)) for t in x], "g-")
    plt.xlim(0, 10)
    plt.ylim(-5, 5)
    plt.grid(True)
    plt.xlabel('t [s]')
    plt.ylabel('$f(t)$')
    plt.axhline(y=0,lw=0.8,color='k')
    plt.axvline(x=0,lw=0.8,color='k')
    with output_plot:
        output_plot.clear_output(wait=True)
        plt.show()

def f_delta(a):
    plt.figure(figsize=(10,5))
    x = np.linspace(-10, 10, num=1000)
    #plt.plot(x, [A * math.exp(-decay * t) *(np.cos(t*frequency + phase) + np.sin(t*frequency + phase)) for t in x])
    plt.plot(x, [1 / (abs(a)*math.sqrt(np.pi)) * np.e **(-(t/a)**2) for t in x], "b-")
    plt.xlim(-10, 10)
    plt.ylim(-3, 6)
    plt.grid(True)
    plt.xlabel('t [s]')
    plt.ylabel('$f(t)$')
    plt.axhline(y=0,lw=0.8,color='k')
    plt.axvline(x=0,lw=0.8,color='k')
    with output_plot:
        output_plot.clear_output(wait=True)
        plt.show()

def f_step(a,b):
    plt.figure(figsize=(10,5))
    step = lambda x, a, b: b if x > a else 0

    x = np.linspace(-10, 10, num=1000)
    #plt.plot(x, [A * math.exp(-decay * t) *(np.cos(t*frequency + phase) + np.sin(t*frequency + phase)) for t in x])
    plt.plot(x, [step(t, a, b) for t in x] , "r-")
    plt.xlim(-10, 10)
    plt.ylim(-3, 6)
    plt.grid(True)
    plt.xlabel('t [s]')
    plt.ylabel('$f(t)$')
    plt.axhline(y=0,lw=0.8,color='k')
    plt.axvline(x=0,lw=0.8,color='k')
    with output_plot:
        output_plot.clear_output(wait=True)
        plt.show()

def f_ramp(a):
    plt.figure(figsize=(10,5))
    step = lambda x, a: x - a if x > a else 0

    x = np.linspace(-10, 10, num=1000)
    #plt.plot(x, [A * math.exp(-decay * t) *(np.cos(t*frequency + phase) + np.sin(t*frequency + phase)) for t in x])
    plt.plot(x, [step(t, a) for t in x] , "g-")
    plt.xlim(-10, 10)
    plt.ylim(-3, 6)
    plt.grid(True)
    plt.xlabel('t [s]')
    plt.ylabel('$f(t)$')
    plt.axhline(y=0,lw=0.8,color='k')
    plt.axvline(x=0,lw=0.8,color='k')
    with output_plot:
        output_plot.clear_output(wait=True)
        plt.show()
    
def f_triangle(a, p):
    plt.figure(figsize=(10,5))
    t = np.linspace(-10, 10, num=1000)
    plt.plot(t, [2 * a / np.pi * np.arcsin(np.sin(2*np.pi * x / p)) for x in t], 'b-')
    plt.xlim(-10, 10)
    plt.ylim(-5, 5)
    plt.grid(True)
    plt.xlabel('t [s]')
    plt.ylabel('$f(t)$')
    plt.axhline(y=0,lw=0.8,color='k')
    plt.axvline(x=0,lw=0.8,color='k')    
    with output_plot:
        output_plot.clear_output(wait=True)
        plt.show()    
    
def first_setup():
    with output_info:
        output_info.clear_output()
        display(Markdown(sine_text))
                
    with output_panel:
        output_panel.clear_output()
        display(Markdown(formula_sine))
        display(interactive(f_sin, A=slider_a, frequency=slider_f, phase=slider_p))   


def dropdown_eventhandler(change):    
    if (dd_order.value == 'szinusz hullám'):
        with output_info:
            output_info.clear_output()
            display(Markdown(sine_text))
        
        with output_panel:
            output_panel.clear_output()
            display(Markdown(formula_sine))
            display(interactive(f_sin, A=slider_a, frequency=slider_f, phase=slider_p))  
    
    if (dd_order.value == 'koszinusz hullám'):
        with output_info:
            output_info.clear_output()
            display(Markdown(cosine_text))
        
        with output_panel:
            output_panel.clear_output()
            display(Markdown(formula_cosine))
            display(interactive(f_cos, A=slider_acos, frequency=slider_fcos, phase=slider_pcos))
        
    if (dd_order.value == 'csillapított hullám'):
        with output_info:
            output_info.clear_output()
            display(Markdown(dumped_text))
        
        with output_panel:
            output_panel.clear_output()
            display(Markdown(formula_damp))
            display(interactive(f_damping, A=slider_adamp, frequency=slider_fdamp, phase=slider_pdamp, decay=slider_d))
    
    if (dd_order.value == 'impulzus függvény'):
        with output_info:
            output_info.clear_output()
            display(Markdown(delta_text))
        
        with output_panel:
            output_panel.clear_output()
            display(Markdown(formula))
            display(interactive(f_delta, a = slider_adelta))
    
    if (dd_order.value == 'egységugrás függvény'):
        with output_info:
            output_info.clear_output()
            display(Markdown(step_text))
        
        with output_panel:
            output_panel.clear_output()
            display(Markdown(formula_step))
            display(interactive(f_step, a = slider_astep, b = slider_bstep))
    
    if (dd_order.value == 'rámpa függvény'):
        with output_info:
            output_info.clear_output()
            display(Markdown(ramp_text))
        
        with output_panel:
            output_panel.clear_output()
            display(Markdown(formula_ramp))
            display(interactive(f_ramp, a = slider_aramp))
    
    if (dd_order.value == 'háromszög hullám'):
        with output_info:
            output_info.clear_output()
            display(Markdown(triang_text))
        
        with output_panel:
            output_panel.clear_output()
            display(Markdown(formula_triangle))
            display(interactive(f_triangle, a = slider_atri, p = slider_ptri))
    
display(dd_order, output_info, widgets.HBox([output_panel, widgets.Label(" "), output_plot]) )
first_setup()
dd_order.observe(dropdown_eventhandler, names='value')

Dropdown(description='Válasszon függvényt:', options=('szinusz hullám', 'koszinusz hullám', 'csillapított hull…

Output(layout=Layout(border='solid black', height='200', padding='5px', width='100%'))